In [ ]:
#Hopefully Generate working PDFs to out/
#Also test local libraries so clean code
import numpy as np
import math
import pandas as pd
import os
import warnings
from astropy.stats import sigma_clip
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy.ma as ma
import copy
from matplotlib import rc
import numpy.ma as ma
import matplotlib as mpl
import sklearn.metrics
from itertools import chain
from gatspy import datasets, periodic
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from numpy.random import default_rng
#import locutils -- works
#import astfunc -- works
obj=4 #3s index no. 
rc('font', family='serif')
rc('mathtext', fontset='cm')
full_file_list = os.listdir('../full') # creates a Python list containing the file paths for every object's time series

In [1]:
#A ******* disaster...

df = pd.read_csv('../csv/file_list.csv')
file_list = df.values.tolist()
#retrieve list of known >20MOM vals
df_l = pd.read_csv('../csv/lsa.csv')
lsa = df_l.values.tolist()
#len(lsa)
#make col 0 an int like it should be
for i in range(len(lsa)):
    del lsa[i][1]
    lsa[i] = int(lsa[i][0])

df_l1 = pd.read_csv('../csv/lsa1.csv')
lsa1 = df_l1.values.tolist()
for i in range(len(lsa1)):
    lsa1[i][0] = int(lsa1[i][0])

#this only works when fnames is just the object name
#fnames = file_list[lsa1[lsa[obj]][0]]

print(lsa1)

df_cat = pd.read_csv('../csv/all_categ_vals.csv')
all_categ_vals = df_cat.values.tolist()
all_categ_sort = [[w,x,y,z] for w,x,y,z in all_categ_vals if (x == 3 and y == 3 and z == 3)]
#Much cleaner way of using lsa
#fnames = (file_list[int(lsa[obj][0])])

[[32, 38.27849655813822], [130, 51.06580522004588], [346, 58.07553136821736], [503, 56.51397319786628], [570, 30.96154521117509], [743, 28.938532160597006], [951, 60.50480608189349], [1041, 37.97296384997651], [1248, 35.64942095539538], [1259, 33.361920078373544], [1308, 29.46477579280008], [1457, 36.31761357099042], [1675, 38.9592690697768], [1747, 47.449974605009615], [1813, 24.858180162752245], [2259, 46.32697228065098], [2433, 43.68927722336131], [2556, 25.09384499087865], [2590, 42.63781487488027], [2629, 33.68090209090419], [2655, 34.965047519056284], [3073, 33.55859238633304], [3258, 24.94178603749639], [3316, 55.92436855323568], [3341, 46.48738021639729], [3650, 36.46933158931351], [3666, 52.6302016485628], [3714, 48.68993851789249], [3759, 30.58763478220377], [3881, 48.40964107247068], [3908, 57.36175839524541], [3963, 37.5486687387193], [4082, 35.953593658753654], [4126, 39.953405466977145], [4633, 30.88872448730432], [5096, 49.27135449614008], [5388, 49.241679492550176], [56

In [2]:
"""
mastlist=[]
for i in range(len(lsa)):
    for j in range(len(lsa1)):
        for k in range(len(file_list)):
            if lsa[i] == lsa1[i] 
"""

'\nmastlist=[]\nfor i in range(len(lsa)):\n    for j in range(len(lsa1)):\n        for k in range(len(file_list)):\n            if lsa[i] == lsa1[i] \n'

In [ ]:
all_mags_narrow = pd.read_csv('../csv/all_mags_narrow.csv')#.values.tolist()
all_mjd_narrow = pd.read_csv('../csv/all_mjd_narrow.csv')#.values.tolist()
all_magerrs_narrow = pd.read_csv('../csv/all_magerrs_narrow.csv')#.values.tolist()
all_mags_narrow.drop('Unnamed: 0',inplace=True,axis=1)
all_mjd_narrow.drop('Unnamed: 0',inplace=True,axis=1)
all_magerrs_narrow.drop('Unnamed: 0',inplace=True,axis=1)
all_mags = all_mags_narrow.values.tolist()
all_mjd = all_mjd_narrow.values.tolist()
all_magerrs = all_magerrs_narrow.values.tolist()
for i in range(len(all_mags)):
    for j in range(len(all_mags[i])):
        all_mags[i][j] = all_mags[i][j][1:len(all_mags[i][j])-1].replace('\n','').split()
        all_mjd[i][j] = all_mjd[i][j][1:len(all_mjd[i][j])-1].replace('\n','').split()
        all_magerrs[i][j] = all_magerrs[i][j][1:len(all_magerrs[i][j])-1].replace('\n','').split()
for i in range(len(all_mags)):
    for j in range(len(all_mags[i])):
        for z in range(len(all_mags[i][j])):
            all_mags[i][j][z] = float(all_mags[i][j][z])
            all_mjd[i][j][z] = float(all_mjd[i][j][z])
            all_magerrs[i][j][z] = float(all_magerrs[i][j][z])

In [ ]:
#generate random values to run light curves with
#rint = locutils.randomgen(lsa, 1)

In [3]:
"""
def load_one_timeseries(file_name):
    timestamps = []
    mags = []
    magerrs = []
    expnums = []
    filters = []

    with open('../full/'+file_name) as data_file:
        first_line = data_file.readline()
        field = (first_line.split())[2]
        for line in data_file:
            if not line.startswith('#'):
                data = line.split()
                timestamps.append(data[0])
                mags.append(data[1])
                magerrs.append(data[2])
                expnums.append(data[3])
                filters.append(data[8])

    timestamps = np.asarray(timestamps, dtype=float)
    mags = np.asarray(mags, dtype=float)
    magerrs = np.asarray(magerrs, dtype=float)
    expnums = np.asarray(expnums, dtype=int)

    int_filters = np.empty(len(filters), dtype=int)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        int_filters[np.where(np.asarray(filters)=='U')[0]] = 0
        int_filters[np.where(np.asarray(filters)=='G')[0]] = 1
        int_filters[np.where(np.asarray(filters)=='R')[0]] = 2
        int_filters[np.where(np.asarray(filters)=='I')[0]] = 3
        int_filters[np.where(np.asarray(filters)=='I2')[0]] = 4
        int_filters[np.where(np.asarray(filters)=='Z')[0]] = 5

    num_entries = len(timestamps)

    return field, timestamps, mags, magerrs, expnums, int_filters, num_entries
"""

"\ndef load_one_timeseries(file_name):\n    timestamps = []\n    mags = []\n    magerrs = []\n    expnums = []\n    filters = []\n\n    with open('../full/'+file_name) as data_file:\n        first_line = data_file.readline()\n        field = (first_line.split())[2]\n        for line in data_file:\n            if not line.startswith('#'):\n                data = line.split()\n                timestamps.append(data[0])\n                mags.append(data[1])\n                magerrs.append(data[2])\n                expnums.append(data[3])\n                filters.append(data[8])\n\n    timestamps = np.asarray(timestamps, dtype=float)\n    mags = np.asarray(mags, dtype=float)\n    magerrs = np.asarray(magerrs, dtype=float)\n    expnums = np.asarray(expnums, dtype=int)\n\n    int_filters = np.empty(len(filters), dtype=int)\n    with warnings.catch_warnings():\n        warnings.simplefilter(action='ignore', category=FutureWarning)\n        int_filters[np.where(np.asarray(filters)=='U')[0]] = 

In [4]:
"""
def load_one_timeseries_field(file_name):
    timestamps = []
    mags = []
    magerrs = []
    expnums = []
    filters = []

    with open('../full/'+file_name) as data_file:
        first_line = data_file.readline()
        field = (first_line.split())[2]
        for line in data_file:
            if not line.startswith('#'):
                data = line.split()
                timestamps.append(data[0])
                mags.append(data[1])
                magerrs.append(data[2])
                expnums.append(data[3])
                filters.append(data[8])

    timestamps = np.asarray(timestamps, dtype=float)
    mags = np.asarray(mags, dtype=float)
    magerrs = np.asarray(magerrs, dtype=float)
    expnums = np.asarray(expnums, dtype=int)

    int_filters = np.empty(len(filters), dtype=int)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        int_filters[np.where(np.asarray(filters)=='U')[0]] = 0
        int_filters[np.where(np.asarray(filters)=='G')[0]] = 1
        int_filters[np.where(np.asarray(filters)=='R')[0]] = 2
        int_filters[np.where(np.asarray(filters)=='I')[0]] = 3
        int_filters[np.where(np.asarray(filters)=='I2')[0]] = 4
        int_filters[np.where(np.asarray(filters)=='Z')[0]] = 5

    num_entries = len(timestamps)

    return field, timestamps, mags, magerrs, expnums, int_filters, num_entries

"""

"\ndef load_one_timeseries_field(file_name):\n    timestamps = []\n    mags = []\n    magerrs = []\n    expnums = []\n    filters = []\n\n    with open('../full/'+file_name) as data_file:\n        first_line = data_file.readline()\n        field = (first_line.split())[2]\n        for line in data_file:\n            if not line.startswith('#'):\n                data = line.split()\n                timestamps.append(data[0])\n                mags.append(data[1])\n                magerrs.append(data[2])\n                expnums.append(data[3])\n                filters.append(data[8])\n\n    timestamps = np.asarray(timestamps, dtype=float)\n    mags = np.asarray(mags, dtype=float)\n    magerrs = np.asarray(magerrs, dtype=float)\n    expnums = np.asarray(expnums, dtype=int)\n\n    int_filters = np.empty(len(filters), dtype=int)\n    with warnings.catch_warnings():\n        warnings.simplefilter(action='ignore', category=FutureWarning)\n        int_filters[np.where(np.asarray(filters)=='U')[

In [5]:
"""
def get_num_measurements(file_list, by_filter=False):
    if by_filter==False:
        num_measurements = np.empty(len(file_list))
        for i, fname in enumerate(file_list):
            _, _, _, _, _, _, num = load_one_timeseries(fname)
            num_measurements[i] = num
    elif by_filter==True:
        num_measurements = np.empty((len(file_list), 6))
        for i, fname in enumerate(file_list):
            _, _, _, _, _, filters, _ = load_one_timeseries(fname)
            for j in range(6):
                num_measurements[i,j] = np.sum(filters==j)
    return num_measurements

  Input In [123]
    """
    ^
SyntaxError: incomplete input



In [6]:
#df = pd.read_csv('../csv/file_list')
#file_list = df.tolist()

#if not file_list:
    #num_measurements = astfunc.get_num_measurements(full_file_list)
num_measurements = get_num_measurements(full_file_list)
file_list = (np.asarray(full_file_list))[num_measurements>15]
num_measurements_by_filter = get_num_measurements(file_list, by_filter=True)
    #num_measurements_by_filter = astfunc.get_num_measurements(file_list, by_filter=True)
file_list = file_list[np.all(num_measurements_by_filter>=15, axis=1)]
"""

  Input In [124]
    """
    ^
SyntaxError: incomplete input



In [ ]:
def data_format (object_number):
    filts = []
    temp_u = ['u' for i in all_mags[object_number][0]]
    temp_g = ['g' for i in all_mags[object_number][1]]
    temp_r = ['r' for i in all_mags[object_number][2]]
    temp_i = ['i' for i in all_mags[object_number][3]]
    temp_z = ['z' for i in all_mags[object_number][4]]
    filts.append(temp_u)
    filts.append(temp_g)
    filts.append(temp_r)
    filts.append(temp_i)
    filts.append(temp_z)
    filts = list(chain.from_iterable(filts))
    mags = list(chain.from_iterable(all_mags[object_number])) #turning 2d array into 1d
    t = list(chain.from_iterable(all_mjd[object_number]))
    #dy = [1 for i in mags]
    dy = list(chain.from_iterable(all_magerrs[object_number]))
    dy = [99 if i == 0 else i for i in dy]
    return t, mags, dy, filts

In [ ]:
t, mags, dy, filts = data_format(obj)
obj = 0
print(obj)
periods = np.linspace(0.1, 1.0, 1000000) # This defines the search range of your period, you can specify it at your will. These are in days.

In [7]:
LS_multi = periodic.LombScargleMultiband(Nterms_base=1, Nterms_band=0)#initiate structure variable
LS_multi.fit(t, mags, dy, filts)#input our data
P_multi = LS_multi.periodogram(periods)#function where input is periods

----------
NameErrorTraceback (most recent call last)
Input In [130], in <cell line: 2>()
      1 LS_multi = periodic.LombScargleMultiband(Nterms_base=1, Nterms_band=0)#initiate structure variable
----> 2 LS_multi.fit(t, mags, dy, filts)#input our data
      3 P_multi = LS_multi.periodogram(periods)

NameError: name 't' is not defined


In [8]:
plt.ion() #turns figure display on
plt.figure()
plt.scatter(periods, P_multi, s = 0.05)
best_period = max(P_multi)
for i in range(len(P_multi)):
    if P_multi[i] == best_period:
        index = i
print(periods[index])

----------
NameErrorTraceback (most recent call last)
Input In [128], in <cell line: 3>()
      1 plt.ion() #turns figure display on
      2 plt.figure()
----> 3 plt.scatter(periods, P_multi, s = 0.05)
      4 best_period = max(P_multi)
      5 for i in range(len(P_multi)):

NameError: name 'P_multi' is not defined


In [ ]:
def folded_light_curve(obj, period, pdf_name):
    """
    Function to create folded light curve
    Parameters
    ---
    objects: int index of object of interest in file_list
    Returns
    ---
    pdf of magnitude vs MJD for all objects
    """
    plt.ioff()
    with PdfPages(pdf_name) as pdf:
        plt.figure(figsize = (9, 12))
        plt.xlabel('Phase')
        plt.ylabel('Magnitude')
        plt.gca().invert_yaxis()
        u_phase = [(i%period)/period for i in all_mjd[obj][0]]
        g_phase = [(i%period)/period for i in all_mjd[obj][1]]
        r_phase = [(i%period)/period for i in all_mjd[obj][2]]
        i_phase = [(i%period)/period for i in all_mjd[obj][3]]
        z_phase = [(i%period)/period for i in all_mjd[obj][4]]
        #f*p^2/length of observations, f is about 0.1, f = phase error for individual cycle
        plt.scatter(u_phase, all_mags[obj][0], s = 5, c = 'blue', label = 'u')
        plt.scatter(g_phase, all_mags[obj][1], s = 5, c = 'green', label = 'g')
        plt.scatter(r_phase, all_mags[obj][2], s = 5, c = 'purple', label = 'r')
        plt.scatter(i_phase, all_mags[obj][3], s = 5, c = 'gold', label = 'i')
        plt.scatter(z_phase, all_mags[obj][4], s = 5, c = 'tab:red', label = 'z')
        plt.legend()
        plt.title(''+ " (" + str(obj) + ")") #TODO: make fnames[obj] work right
        pdf.savefig()
        plt.close()
    #0.6 seconds ideal step

In [9]:
    #0.6 seconds ideal step
    # i/p_true + n where n is an integer all reciprocated is the beat frequency
#for i in lsa:
#        obj = lsa[i]
#        print(lsa[i])
folded_light_curve(obj, periods[index], 'out/'+str(obj)+'.pdf')
        print('out/'+str(obj)+'.pdf generated.')

----------
NameErrorTraceback (most recent call last)
Input In [94], in <cell line: 4>()
      4 for i in range(len(lsa)):
      5     obj = lsa[i]
----> 6     folded_light_curve(obj, periods[index], 'out/'+str(obj)+'.pdf')
      7     print('out/'+str(obj)+'.pdf generated.')

NameError: name 'index' is not defined
